In [2]:
using Test, pulse_input_DDM, LinearAlgebra, Flatten, Parameters

In [3]:
n, cross = 53, false

(53, false)

In [4]:
## Choice model
θ = θchoice(θz=θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 50., σ2_s = 1.5,
    ϕ = 0.8, τ_ϕ = 0.05),
    bias=1., lapse=0.05)

θ, data = synthetic_data(;θ=θ, ntrials=10, rng=1)
model_gen = choiceDDM(θ, data, n, cross)

choiceDDM{θchoice{θz{Float64},Float64},Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}},1}}
  θ: θchoice{θz{Float64},Float64}
  data: Array{pulse_input_DDM.choicedata{pulse_input_DDM.choiceinputs{pulse_input_DDM.clicks,pulse_input_DDM.binned_clicks}}}((10,))
  n: Int64 53
  cross: Bool false


In [5]:
choices = getfield.(data, :choice);

In [6]:
@test all(choices .== vcat(falses(9), trues(1)))

Test Passed

In [7]:
@time @test round(loglikelihood(model_gen), digits=2) ≈ -3.72

  1.838436 seconds (4.14 M allocations: 224.753 MiB, 6.72% gc time)


Test Passed

In [8]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 14.32

Test Passed

In [9]:
model, = optimize(data, choiceoptions(); iterations=5, outer_iterations=1);
@test round(norm(Flatten.flatten(model.θ)), digits=2) ≈ 25.05

Fminbox
-------
Initial mu = 0.000239451

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.000239451

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.615269e+00     9.828961e+01
 * time: 7.390975952148438e-5

Exiting inner optimizer with x = [0.12345655322552201, 14.998955306903198, -0.13726257383761056, 19.99805424621316, 0.6951082089315415, 1.1588961674105702, 0.0063088058501644005, 0.6814442212446054, 0.5003949673659975]
Current distance to box: 0.00130881
Decreasing barrier term μ.



Test Passed

In [ ]:
H = Hessian(model)
@test round(norm(H), digits=2) ≈ 92.6

In [ ]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 1471.54

In [ ]:
## Neural model
ncells, ntrials = [1,2], [10,5]
f = [repeat(["Sigmoid"], N) for N in ncells]
                    
θ = θneural(θz = θz(σ2_i = 0.5, B = 15., λ = -0.5, σ2_a = 10., σ2_s = 1.2,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid() for n in 1:N] for N in ncells], f=f);

In [ ]:
data, = synthetic_data(θ, ntrials, ncells);
model_gen = neuralDDM(θ, data, n, cross);

In [ ]:
spikes = map(x-> sum.(x), getfield.(vcat(data...), :spikes))

In [ ]:
@test all(spikes .== [[3], [14], [6], [1], [5], [9], [20], [9], [5], [21], [11, 10], [5, 8], [7, 5], [10, 11], [11, 9]])

In [ ]:
@test round(loglikelihood(model_gen), digits=2) ≈ -529.86

In [ ]:
@test round(norm(gradient(model_gen)), digits=2) ≈ 52.18

In [16]:
x = pulse_input_DDM.flatten(θ)
@test round(loglikelihood(x, model_gen), digits=2) ≈ -529.86

Test Passed

In [17]:
θy0 = vcat(vcat(θy.(data, f)...)...)
@test round(norm(θy0), digits=2) ≈ 31.16

Test Passed

In [18]:
#deterministic model
options0 = neural_options_noiseless(f)

neural_options_noiseless
  fit: Array{Bool}((19,)) Bool[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  ub: Array{Float64}((19,)) [100.0, 100.0, 5.0, 400.0, 10.0, 1.2, 1.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0, 100.0, 100.0, 10.0, 10.0]
  lb: Array{Float64}((19,)) [0.001, 8.0, -5.0, 0.001, 0.001, 0.001, 0.005, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0, -100.0, 0.0, -10.0, -10.0]


In [19]:
x0=vcat([0., 30., 0. + eps(), 0., 0., 1. - eps(), 0.008], θy0)

19-element Array{Float64,1}:
   0.0                  
  30.0                  
   2.220446049250313e-16
   0.0                  
   0.0                  
   0.9999999999999998   
   0.008                
  -7.705663789255116    
   7.678951595360267    
   0.0011352713910126341
   0.0                  
 -11.526171720551112    
  11.655646461267072    
   0.008735402516937699 
   0.0                  
  -8.170615489116662    
  22.738475419974247    
   0.0075096756861132   
   0.0                  

In [20]:
θ0 = θneural_noiseless(x0, f)
model0 = noiseless_neuralDDM(θ0, data)

noiseless_neuralDDM{θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}},Array{Array{neuraldata,1},1}}
  θ: θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  data: Array{Array{neuraldata,1}}((2,))


In [21]:
@test round(loglikelihood(model0), digits=2) ≈ -1399.68

Test Passed

In [22]:
x0 = pulse_input_DDM.flatten(θ0)
@unpack f = θ0

θneural_noiseless{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Sigmoid{Float64},1}}((2,))
  f: Array{Array{String,1}}((2,))


In [23]:
@test round(loglikelihood(x0, model0), digits=2) ≈ -1399.68

Test Passed

In [27]:
model, = optimize(model0, options0; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 87.51

Test Passed

In [28]:
@test round(norm(gradient(model)), digits=2) ≈ 6.82

Test Passed

In [29]:
x0 = vcat([0.1, 15., -0.1, 20., 0.5, 0.8, 0.008], pulse_input_DDM.flatten(model.θ)[dimz+1:end])

19-element Array{Float64,1}:
  0.1               
 15.0               
 -0.1               
 20.0               
  0.5               
  0.8               
  0.008             
 52.69070630256158  
 43.434427853058395 
 -2.8625905261748006
  5.0336085210607395
 17.047935166694103 
 33.8056973114395   
  5.1937297903733795
  3.97561896305667  
 -5.690347702664896 
 23.203791176447094 
  0.4792342779693217
  3.6141666197326594

In [30]:
options = neural_options(f)  

model = neuralDDM(θneural(x0, f), data, n, cross)
model, = optimize(model, options; iterations=2, outer_iterations=1)
@test round(norm(pulse_input_DDM.flatten(model.θ)), digits=2) ≈ 85.73

Fminbox
-------
Initial mu = 0.00248599

Fminbox iteration 1
-------------------
Calling inner optimizer with mu = 0.00248599

(numbers below include barrier contribution)
Iter     Function value   Gradient norm 
     0     7.682407e+02     7.356733e+02
 * time: 3.409385681152344e-5

Exiting inner optimizer with x = [1.051828368475773, 15.03668934024119, 0.2797507123339492, 20.04006641872528, 0.5926364853980968, 0.01838739503368525, 0.7082387560405169, 52.42451557332407, 43.40460710623912, -3.3663760919374215, 4.683811149066144, 16.995759054681873, 33.792099189665144, 4.917694876411467, 3.9073304947867564, -5.689492847672851, 23.20298505742793, 0.280912565861838, 3.6334905003779103]
Current distance to box: 0.0173874
Decreasing barrier term μ.



Test Passed

In [31]:
H = Hessian(model; chuck_size=4)
@test round(norm(H), digits=2) ≈ 22.83

Test Passed

In [33]:
CI, HPSD = CIs(H)
@test round(norm(CI), digits=2) ≈ 303.03

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 1.7824957110747937
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


Test Passed